# **`robopianist` tutorial**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/google-research/robopianist/blob/main/tutorial.ipynb)








> <p><small><small>Copyright 2023 The RoboPianist Authors.</small></p>
> <p><small><small>Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at <a href="http://www.apache.org/licenses/LICENSE-2.0">http://www.apache.org/licenses/LICENSE-2.0</a>.</small></small></p>
> <p><small><small>Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.</small></small></p>

# Installing `robopianist` on Colab

In [ ]:
# @title Run to install robopianist
from IPython.display import clear_output
import subprocess

if subprocess.run("nvidia-smi").returncode:
    raise RuntimeError(
        "Cannot communicate with GPU. "
        "Make sure you are using a GPU Colab runtime. "
        "Go to the Runtime menu and select Choose runtime type."
    )

# Install dependencies.
# %shell bash <(curl -s https://raw.githubusercontent.com/google-research/robopianist/main/scripts/install_deps.sh) --no-soundfonts
%git clone https://github.com/google-research/robopianist.git && cd robopianist
%git submodule init && git submodule update
%shell bash scripts/install_deps.sh
print("Installing robopianist...")
# %pip install -q robopianist>=1.0.6
%pip install -e ".[dev]"
%pip install stable-baselines3[extra]
%env MUJOCO_GL=egl

clear_output()
%shell echo Installed $(robopianist --version)

# Imports

In [1]:
# @title All imports required for this tutorial
from IPython.display import HTML
from base64 import b64encode
import numpy as np
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from robopianist.suite.tasks import self_actuated_piano
from robopianist.suite.tasks import piano_with_shadow_hands
from dm_env_wrappers import CanonicalSpecWrapper
from robopianist.wrappers import PianoSoundVideoWrapper
from robopianist import music
from mujoco_utils import composer_utils

import dm_env

In [6]:
# @title Helper functions


# Reference: https://stackoverflow.com/a/60986234.
def play_video(filename: str):
    mp4 = open(filename, "rb").read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

    return HTML(
        """
  <video controls>
        <source src="%s" type="video/mp4">
  </video>
  """
        % data_url
    )

import dm_env
from dm_env import specs
from gym import spaces
import numpy as np
from dm_env import specs


def convert_dm_control_to_gym_space(dm_control_space):
    r"""Convert dm_control space to gym space. """
    if isinstance(dm_control_space, specs.BoundedArray):
        space = spaces.Box(low=dm_control_space.minimum, 
                           high=dm_control_space.maximum, 
                           dtype=dm_control_space.dtype)
        assert space.shape == dm_control_space.shape
        return space
    elif isinstance(dm_control_space, specs.Array) and not isinstance(dm_control_space, specs.BoundedArray):
        space = spaces.Box(low=-float('inf'), 
                           high=float('inf'), 
                           shape=dm_control_space.shape, 
                           dtype=dm_control_space.dtype)
        return space
    elif isinstance(dm_control_space, dict):
        space = spaces.Dict({key: convert_dm_control_to_gym_space(value)
                             for key, value in dm_control_space.items()})
        return space

class DMSuiteEnv(gym.Env):
    def __init__(self, env):
        self.env = env 
        self.wrapped = env
        self.metadata = {'render.modes': ['human', 'rgb_array'],
                         'video.frames_per_second': round(1.0/self.env.control_timestep())}

        self.observation_space = convert_dm_control_to_gym_space(self.env.observation_spec())
        self.action_space = convert_dm_control_to_gym_space(self.env.action_spec())
        self.viewer = None
    
    def seed(self, seed):
        return self.env.task.random.seed(seed)
    
    def step(self, action):
        timestep = self.env.step(action)
        observation = timestep.observation
        reward = timestep.reward
        done = timestep.last()
        info = {}
        return observation, reward, done, info
    
    def reset(self):
        timestep = self.env.reset()
        return timestep.observation
    
    def render(self, mode='human', **kwargs):
        if 'camera_id' not in kwargs:
            kwargs['camera_id'] = 0  # Tracking camera
        use_opencv_renderer = kwargs.pop('use_opencv_renderer', False)
        
        img = self.env.physics.render(**kwargs)
        if mode == 'rgb_array':
            return img
        elif mode == 'human':
            if self.viewer is None:
                if not use_opencv_renderer:
                    from gym.envs.classic_control import rendering
                    self.viewer = rendering.SimpleImageViewer(maxwidth=1024)
                else:
                    from . import OpenCVImageViewer
                    self.viewer = OpenCVImageViewer()
            self.viewer.imshow(img)
            return self.viewer.isopen
        else:
            raise NotImplementedError

    def close(self):
        if self.viewer is not None:
            self.viewer.close()
            self.viewer = None
        return self.env.close()

# Self-actuated piano task

In [4]:
task = self_actuated_piano.SelfActuatedPiano(
    midi=music.load("TwinkleTwinkleRousseau"),
    change_color_on_activation=True,
    trim_silence=True,
    control_timestep=0.01,
)

env = composer_utils.Environment(
    recompile_physics=False, task=task, strip_singleton_obs_buffer_dim=True
)

env = PianoSoundVideoWrapper(
    env,
    record_every=1,
    camera_id="piano/back",
    record_dir=".",
)

fluidsynth: warning: SDL2 not initialized, SDL2 audio driver won't be usable


In [5]:
action_spec = env.action_spec()
min_ctrl = action_spec.minimum
max_ctrl = action_spec.maximum
print(f"Action dimension: {action_spec.shape}")

Action dimension: (89,)


In [6]:
print("Observables:")
timestep = env.reset()
dim = 0
for k, v in timestep.observation.items():
    print(f"\t{k}: {v.shape} {v.dtype}")
    dim += np.prod(v.shape)
print(f"Observation dimension: {dim}")

Observables:
	goal: (89,) float64
	piano/activation: (88,) float64
	piano/sustain_activation: (1,) float64
Observation dimension: 178


In [6]:
class Oracle:
    def __call__(self, timestep: dm_env.TimeStep) -> np.ndarray:
        if timestep.reward is not None:
            assert timestep.reward == 0
        # Only grab the next timestep's goal state.
        goal = timestep.observation["goal"][: task.piano.n_keys]
        key_idxs = np.flatnonzero(goal)
        # For goal keys that should be pressed, set the action to the maximum
        # actuator value. For goal keys that should be released, set the action to
        # the minimum actuator value.
        action = min_ctrl.copy()
        action[key_idxs] = max_ctrl[key_idxs]
        # Grab the sustain pedal action.
        action[-1] = timestep.observation["goal"][-1]
        return action

In [7]:
policy = Oracle()

timestep = env.reset()
while not timestep.last():
    action = policy(timestep)
    timestep = env.step(action)

In [8]:
play_video(env.latest_filename)

# Piano with Shadow Hands

In [7]:
task = piano_with_shadow_hands.PianoWithShadowHands(
    change_color_on_activation=True,
    midi=music.load("TwinkleTwinkleRousseau"),
    trim_silence=True,
    control_timestep=0.05,
    gravity_compensation=True,
    primitive_fingertip_collisions=False,
    reduced_action_space=False,
    n_steps_lookahead=10,
    disable_fingering_reward=False,
    disable_forearm_reward=False,
    disable_colorization=False,
    disable_hand_collisions=False,
    attachment_yaw=0.0,
)

composer_env = composer_utils.Environment(
    task=task, strip_singleton_obs_buffer_dim=True, recompile_physics=False
)

wrapper_env = PianoSoundVideoWrapper(
    composer_env,
    record_every=1,
    camera_id="piano/back",
    record_dir=".",
)

wrapper_env = CanonicalSpecWrapper(wrapper_env)

fluidsynth: warning: SDL2 not initialized, SDL2 audio driver won't be usable


In [6]:
action_spec = wrapper_env.action_spec()
print(f"Action dimension: {action_spec.shape}")

Action dimension: (45,)


In [7]:
timestep = wrapper_env.reset()
dim = 0
for k, v in timestep.observation.items():
    print(f"\t{k}: {v.shape} {v.dtype}")
    dim += int(np.prod(v.shape))
print(f"Observation dimension: {dim}")

	goal: (979,) float64
	fingering: (10,) float64
	piano/joints_vel: (88,) float64
	piano/state: (88,) float64
	piano/sustain_state: (1,) float64
	piano/vel_state: (88,) float64
	rh_shadow_hand/joints_pos: (26,) float64
	rh_shadow_hand/position: (3,) float64
	lh_shadow_hand/joints_pos: (26,) float64
	lh_shadow_hand/position: (3,) float64
Observation dimension: 1312


In [11]:
# Download pretrained policy action sequence.
!wget https://github.com/google-research/robopianist/raw/main/examples/twinkle_twinkle_actions.npy > /dev/null 2>&1


class Policy:
    def __init__(self) -> None:
        self.reset()

    def reset(self) -> None:
        self._idx = 0
        self._actions = np.load("twinkle_twinkle_actions.npy")

    def __call__(self, timestep: dm_env.TimeStep) -> np.ndarray:
        del timestep  # Unused.
        actions = self._actions[self._idx]
        self._idx += 1
        return actions

In [12]:
policy = Policy()

timestep = wrapper_env.reset()
while not timestep.last():
    action = policy(timestep)
    timestep = wrapper_env.step(action)

In [13]:
play_video(wrapper_env.latest_filename)

# Training


In [15]:
# If your env does not already output a single dict as observation
# stable_env = GymFromDMEnv(composer_env)
stable_env = DMSuiteEnv(composer_env)
stable_env = DummyVecEnv([lambda: stable_env])
# Wrap it in a vectorized environment
# Initialize the agent
model = PPO('MultiInputPolicy', env=stable_env, verbose=2,tensorboard_log="./robopianist_ppo_tensorboard/")
print(model.get_env())
# Train the agent
model.learn(total_timesteps=5000)

# Save the model
model.save("ppo_robopianist")

# Load the trained model
model = PPO.load("ppo_robopianist")

# Evaluate the policy
mean_reward, std_reward = evaluate_policy(model, stable_env, n_eval_episodes=1)

print(f"mean_reward:{mean_reward:.2f} +/- {std_reward:.2f}")

Using cpu device
Logging to ./robopianist_ppo_tensorboard/PPO_8
-----------------------------
| time/              |      |
|    fps             | 88   |
|    iterations      | 1    |
|    time_elapsed    | 23   |
|    total_timesteps | 2048 |
-----------------------------
---------------------------------------
| time/                   |           |
|    fps                  | 82        |
|    iterations           | 2         |
|    time_elapsed         | 49        |
|    total_timesteps      | 4096      |
| train/                  |           |
|    approx_kl            | 0.0435396 |
|    clip_fraction        | 0.352     |
|    clip_range           | 0.2       |
|    entropy_loss         | -63.9     |
|    explained_variance   | -0.0494   |
|    learning_rate        | 0.0003    |
|    loss                 | 12.9      |
|    n_updates            | 10        |
|    policy_gradient_loss | -0.0896   |
|    std                  | 1         |
|    value_loss           | 69.5      |
------

/home/hmturner/miniconda3/envs/pianist/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward:200.06 +/- 0.00
